### Imports

In [3]:
import copy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

### Suppresses Warning
import warnings
warnings.filterwarnings('ignore')

seed = 42
random.seed(seed)
np.random.seed(seed)

### Functions

In [4]:
def get_min_max_combinations(ranges_dict, feature):
    
    # min of all other features, max of this feature
    min_combo = []
    for key in ranges_dict.keys():
        
        if key == feature:
            min_combo.append(min(ranges_dict[key]))
        else:
            min_combo.append(max(ranges_dict[key]))
        

    # max of all other features, min of this feature
    max_combo = []
    for key in ranges_dict.keys():
        
        if key == feature:
            max_combo.append(max(ranges_dict[key]))
        else:
            max_combo.append(min(ranges_dict[key]))
    
    return tuple(min_combo), tuple(max_combo)

In [5]:
def get_index(df, combo):
    
    # split into components
    Mg_glutamate, K_glutamate, Amino_Acid, tRNA, coA, NAD, cAMP, Folinic_Acid, Spermidine, Three_PGA, NTP = combo
    
    # get index of matching row in data
    a = df.index[
              np.isclose(df['Mg-glutamate (mM)'], Mg_glutamate) \
            & np.isclose(df['K-glutamate (mM)'], K_glutamate) \
            & np.isclose(df['Amino Acid (mM)'], Amino_Acid) \
            & np.isclose(df['tRNA (mg/ml)'], tRNA) \
            & np.isclose(df['coA (mM)'],coA)  \
            & np.isclose(df['NAD (mM)'], NAD) \
            & np.isclose(df['cAMP (mM)'], cAMP) \
            & np.isclose(df['Folinic Acid (mM)'], Folinic_Acid) \
            & np.isclose(df['Spermidine (mM)'], Spermidine) \
            & np.isclose(df['3-PGA (mM)'], Three_PGA) \
            & np.isclose(df['NTP (mM)'], NTP)]
    return a.tolist()[0]



In [6]:
def get_initial_22_idx(df, ranges_dict):
    
    initial_idx = []
    
    # for each feature
    for key in ranges_dict.keys():
        min_combo, max_combo = get_min_max_combinations(ranges_dict,key)
        idx = get_index(df, min_combo)
        initial_idx.append(idx)
        idx = get_index(df, max_combo)
        initial_idx.append(idx)
        
    return np.array(initial_idx)
    

# Data Prep

### Load Data

In [7]:
data = pd.read_csv('data/DataPool.csv')
print(data.shape)
data.head(5)
cols = data.columns

(1017, 12)


### Split into features and target

In [8]:
X = data.drop(['Yield'],axis=1)
print(X.shape)
y_pool = data['Yield'].to_numpy()
print(y_pool.shape)

(1017, 11)
(1017,)


### Scale data
So all variables have the same mean and std dev

In [9]:
scaler = StandardScaler()
X_pool = scaler.fit_transform(X)

# Define initial dataset
- 11 data points in which each compound is minimized except one is maximized
- 11 data pionts in which each compound is maximized except one is minimized
- 80 random data points

In [10]:
ranges_dict = {
    'Mg-glutamate (mM)':[0.4,1.2,2,4],
    'K-glutamate (mM)':[8,24,40,80],
    'Amino Acid (mM)': [0.15,0.45,0.75,1.5],
    'tRNA (mg/ml)':[0.02,0.06,0.1,0.2],
    'coA (mM)':[0.026,0.078,0.13,0.26],
    'NAD (mM)':[0.033,0.099,0.165,0.33],
    'cAMP (mM)':[0.075,0.225,0.375,0.75],
    'Folinic Acid (mM)':[0.0068,0.0204,0.034,0.068],
    'Spermidine (mM)':[0.1,0.3,0.5,1],
    '3-PGA (mM)':[3,9,15,30],
    'NTP (mM)':[0.15,0.45,0.75,1.5]
}


In [11]:
idx_first_22 = get_initial_22_idx(data, ranges_dict)

X_training_22 = X_pool[idx_first_22]
y_training_22 = y_pool[idx_first_22]
print(X_training_22.shape)
print(y_training_22.shape)

# remove from pool
X_pool = np.delete(X_pool, idx_first_22, axis=0)
y_pool = np.delete(y_pool, idx_first_22)   
print(X_pool.shape)
print(y_pool.shape)

# chose 80 points from pool at random
idx_random_80 = np.random.choice(range(len(X_pool)), size=80, replace=False)
X_training_80, y_training_80 = X_pool[idx_random_80], y_pool[idx_random_80]
print(X_training_80.shape)
print(y_training_80.shape)

# remove 80 points from pool
X_pool = np.delete(X_pool, idx_random_80, axis=0)
y_pool = np.delete(y_pool, idx_random_80)   
print(X_pool.shape)
print(y_pool.shape) 

# combine training sets together
X_training = np.append(X_training_22, X_training_80, axis=0)
y_training = np.append(y_training_22, y_training_80, axis=0)
print(X_training.shape)
print(y_training.shape)

assert X_training.shape == (102,11)
assert y_training.shape == (102, )

(22, 11)
(22,)
(995, 11)
(995,)
(80, 11)
(80,)
(915, 11)
(915,)
(102, 11)
(102,)


# Define Model